In [1]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates


from QUANTAXIS.QAUtil import DATABASE
from QUANTAXIS.QAUtil import  trade_date_sse
from QUANTAXIS.QAUtil.QADate_trade import (
    QA_util_get_pre_trade_date,
    QA_util_get_next_trade_date,
    QA_util_if_tradetime
)

import ind.alpha191 as a191
from scipy import stats
import statsmodels.api as sm

# import cpuinfo
# if 'ntel' in cpuinfo.get_cpu_info()['brand_raw']:
# from sklearnex import patch_sklearn, unpatch_sklearn
# unpatch_sklearn() ##注意，少量数据的线性回归没有优势。慎用，存在内存泄露

from sklearn import linear_model


%load_ext autoreload
%autoreload 2
%aimport ind.alpha191


In [2]:
def pretreate_data(data):
    returns = smpl.get_current_return(data,'close')
    returns.name = 'returns'
    ret_forward = smpl.get_forward_return(data,'close')
    ret_forward.name = 'ret_forward'
    # {'Open', 'cap', 'close', 'high', 'ind', 'low', 'returns', 'volume', 'vwap'}
    data = pd.concat([data, returns, ret_forward], axis=1)
    data = data.assign(vwap=data.amount/(data.volume*100))
    data = smpl.add_marketvalue_industry(data)
    data.rename(columns = {"open":"Open",'totalCapital':'cap','industry':'ind'}, inplace=True)
    data = smpl.add_report_inds(data,'netAssetsPerShare')
    data['pb']=data['close']/data['netAssetsPerShare']
    
    df_ids = data.index.get_level_values(0)
    
    benchmark = smpl.get_benchmark('沪深300', start=df_ids.min(), end=df_ids.max()).data
    benchmark = benchmark[['open','close']].reset_index('code',drop=True)
    data['bm_index_open'] = df_ids.map(benchmark['open'])
    data['bm_index_close'] = df_ids.map(benchmark['close'])
    return data
    # return pd.concat([data, close_ind, vwap_ind, low_ind, high_ind, volume_ind, adv20_ind, adv40_ind, adv81_ind, co_mixed_ind, oh_mixed_ind, lv_mixed_ind], axis=1)


In [3]:
def assemble_stocks_by_codes(codes):
    files = list(map(lambda x:x+'_train_qfq',codes))
    return pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),files))).sort_index(level=0)

l = smpl.get_codes_from_blockname('沪深300', sse='all')
df_all =  assemble_stocks_by_codes(l)

In [4]:
df_treated = pretreate_data(df_all)

In [5]:
# ids = [11,24,38,41,42,47,57,69,80,82,83,88,93,97]
# ids = [92,157,149,191]

# ['000001','601728','000002','601825', '601868']
ids = np.arange(1,192)
np.random.shuffle(ids)

def generate_alpha_factors(fun_ids, df=None):
    import ind.alpha191 as a191
    from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
    from base.JuUnits import excute_for_multidates
    import numpy as np

    for i in fun_ids:
    # for i in ids:
        fun_name = 'alpha'+ '0'*(3- int(np.log10(i)+1)) +str(i)
        if fun_name != 'alpha030':
            indx = excute_for_multidates(df, lambda x: getattr(a191, fun_name)(x) ,level=1)
        else:
            indx = getattr(a191, fun_name)(df)
        # indx = excute_for_multidates(df_treated.loc[(slice(None),['000001','601728','000002','601825', '601868']),:], lambda x: getattr(a191, fun_name)(x) ,level=1)
        indx.name = '191_'+fun_name
        save_cache('{}_train'.format(indx.name), indx, cache_type=CACHE_TYPE.FACTOR)
        # print(indx)

worker=6
task = task_chunk_split(ids, worker)
results = parallal_task(worker, generate_alpha_factors, task, df=df_treated)


Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:7744  pid:23192


  0%|          | 0/6 [00:00<?, ?it/s]

In [51]:
# excute_for_multidates(df_treated.sort_index(), lambda x: a191.alpha011(x) ,level=1)
def STD(data, windows):
    return data.rolling(window=windows, min_periods=windows).std()
def MEAN(data, windows):
    return data.rolling(window=windows, min_periods=windows).mean()
def DELTA(data, windows):
    return data.diff(4)
def SEQUENCE(n):
    return pd.Series(np.arange(1,n+1))

def SMA(data,windows,alpha):
    return data.ewm(adjust=False, alpha=float(alpha)/windows, min_periods=windows, ignore_na=False).mean()

def REGBETA(xs, y, n):
    assert len(y)>=n,  'len(y)!>=n !!!'+ str(y.index[0])
    regress = linear_model.LinearRegression(fit_intercept=False)
    def reg(X,Y):
        try:
            if len(Y)>len(X):
                Y_ =  Y[X.index]
                if Y_.isnull().any():
                    return np.nan
                res = regress.fit(X.values.reshape(-1, 1), Y_.values.reshape(-1, 1)).coef_[0]
            else:
                # if Y.isnull().any():
                #     return np.nan
                res = regress.fit(X.values.reshape(-1, 1), Y.values.reshape(-1, 1)).coef_[0]
        except Exception as e:
            print(e)
            return np.nan
        return res
    return xs.rolling(window=n, min_periods=n).apply(lambda x:reg(x,y))


def COVIANCE(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].cov(B.iloc[x]))
    return se

def CORR(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].corr(B.iloc[x]))
    return se





In [50]:
# df_treated.index.get_level_values(1).unique().to_list()
# '601728',
#  '601825',
#  '601868']
# alpha149(df_treated.loc[(slice(None),['601825']),:])

a = df_treated#.loc[(slice(None),['000001','000002']),:]
# a = df_treated.loc[(pd.Timestamp('2017-01-05'),['000001','000002']),:]
# a = df_treated.loc[([pd.Timestamp('2017-01-05'),pd.Timestamp('2017-01-06')],['000001', '000002', '000063', '000066', '000069', '000100', '000157','000166', '000301', '000333']),:]
# a = df_treated.loc[([pd.Timestamp('2017-01-05'),pd.Timestamp('2017-01-06')],['000001', '000002', '000063', '000066', '000069', '000100', '000157','000166', '000301', '000333']),:]

# d_ = d.dropna(subset=['returns'],axis=0)

def alpha186(data, dependencies=['low','high','close'], max_window=20):
    # 就是ADXR
#     (MEAN(ABS(SUM((LD>0 & LD>HD)?LD:0,14)*100/SUM(TR,14) - SUM((HD>0 & HD>LD)?HD:0,14)*100/SUM(TR,14)) 
#     /
#     (SUM((LD>0  &  LD>HD)?LD:0,14)*100/SUM(TR,14) + SUM((HD>0 & HD>LD)?HD:0,14)*100/SUM(TR,14))*100,6)
#     +
#     DELAY(MEAN(ABS(SUM((LD>0 & LD>HD)?LD:0,14)*100/SUM(TR,14) - SUM((HD>0 & HD>LD)?HD:0,14)*100/SUM(TR,14)) 
#            / (SUM((LD>0 & LD>HD)?LD:0,14)*100/SUM(TR,14) + SUM((HD>0 & HD>LD)?HD:0,14)*100/SUM(TR,14))*100,6)
#          ,6))/2
    dm_plus = data['close'].diff(1).fillna(0)
    dm_subtract = data['low'].diff(1).fillna(0)
    condition_plus = (dm_plus<dm_subtract) | (dm_plus<0)
    condition_sub = (dm_subtract < dm_plus) | (dm_subtract<0)
    dm_plus[condition_plus] = 0
    dm_subtract[condition_sub] = 0
    
    close_delay = data['close'].shift(1)
    tr_a = data['high'] - data['low']
    tr_b = data['high'] - close_delay
    tr_c = data['low'] - close_delay
    tr = pd.concat([tr_a,tr_b,tr_c],axis=1).max(axis=1)
    
    tr_sum = tr.rolling(14).sum()
    
    PDI = dm_plus.rolling(14).sum() * 100 / tr_sum
    MDI = dm_subtract.rolling(14).sum() * 100 / tr_sum
    DX = np.abs(PDI-MDI)/(PDI+MDI) * 100
    ADX = MEAN(DX,6)
    ADXR =(ADX+ADX.shift(1))/2
    
    return ADXR


# with pd.option_context('display.max_rows', None): 
display(excute_for_multidates(df_treated.loc[(slice(None),['000001', '000002', '000063', '000066', '000069', '000100', '000157','000166', '000301', '000333']),:], lambda x: alpha186(x) ,level=1))


date        code  
2017-01-04  000001          NaN
2017-01-05  000001          NaN
2017-01-06  000001          NaN
2017-01-09  000001          NaN
2017-01-10  000001          NaN
                        ...    
2021-12-24  000333    13.081014
2021-12-27  000333    13.642905
2021-12-28  000333    15.457109
2021-12-29  000333    14.600930
2021-12-30  000333    11.121865
Length: 11792, dtype: float64